# Goal of this notebook

The goal of this notebook is to work through BIP 340 in full mathematical detail in order to help me understand it.
Also if I ever go back to BIP 340 in the future I'll have a deeper reference here than the source BIP PR.

Before reading BIP 340, I highly recommend understanding elliptic curve groups and the original ECDSA algorithm first.
This repository has another notebook named `elliptic_curves` with a tutorial there.

# Schnorr Signature Basics

Suppose Alice has a private/public key pair $d_A$, $Q_A$ and wishes to sign a message hash $m$.
Recall the original ECDSA algorithm:

Sign:
1. Generate random private nonce $k$
2. Calculate public nonce $R=k \times G$
3. Calculate $r=\text{X-coordinate}(R)$
3. Calculate $s=(m + r \cdot d_A) \cdot k^{-1}$
4. Broadcast $(r,s)$

Verify:
1. Calculate $u_1 = m \cdot s^{-1}$
2. Calculate $u_2 = r \cdot s^{-1}$
3. Calculate $R' = u_1 \times G + u_2 \times Q_A$
4. Calculate $r'=\text{X-coordinate}(R')$
5. Check that $r'=r$

The Schnorr signature algorithm is also over elliptic curves, but is even simpler than ECDSA. The Schnorr algorithm works as follows:

Sign:
1. Generate random private nonce $k$
2. Calculate public nonce $R=k \times G$
3. Calculate $s = k + \text{Hash}(R||m) \cdot d_A$
4. Broadcast $(R,s)$

Verify:
1. Calculate $P = s \times G$
2. Calculate $P' = R + \text{Hash}(R||m) \times Q_A$
3. Check that $P=P'$

This is trivial to check for correctness, since $R=k \times G$ and $Q_A = d_A \times G$. It is also secure since there is no way for an attacker to easily split $s$ into its additive components $k$ and $\text{Hash}(R||m) \cdot d_A$.

## Schnorr n-of-n Multi-Sig

One of the neat things about Schnorr is that because it is based on addition, it provides n-of-n multi-sig "for free" simply by adding the individual public keys $Q_i$ and signatures $R_i$ and $s_i$. Since the resulting $Q$, $R$, and $s$ are just another key/sig, it looks identical to a single-sig, resulting in shorter scripts and more privacy. Suppose Alice and Bob are constructing and signing a 2-of-2 multi-sig, then it works as follows.

1. Alice constructs a private/public key pair $d_A$, $Q_A$
2. Bob constructs a private/public key pair $d_B$, $Q_B$
3. Alice and Bob share their individual public keys with each other and derive $Q=Q_A + Q_B$
4. They use $Q$ as the address to send funds

Sign:
1. Alice generates a random private nonce $k_A$ and calculates $R_A=k_A \times G$
2. Bob generates a random private nonce $k_B$ and calculates $R_B=k_B \times G$
3. Alice shares $R_A$ with Bob and vice-versa Bob shares $R_B$ with Alice
4. Alice and Bob both calculate $R=R_A + R_B$
5. Alice calculates $s_A = k_A + \text{Hash}(R||m)$
6. Bob calculates $s_B = k_B + \text{Hash}(R||m)$
7. Alice shares $s_A$ with Bob or vice-versa Bob shares $s_B$ with Alice
8. Alice or Bob calculates $s=s_A + s_B$ and broadcasts (R,s)

Verify:
1. Calculate $P = s \times G$
2. Calculate $P' = R + \text{Hash}(R||m) \times Q$
3. Check that $P=P'$

It works because:
\begin{align*}
P &= s \times G \\
&= (s_A + s_B) \times G \\
&= s_A \times G + s_B \times G \\
P' &= R + \text{Hash}(R||m) \times Q \\
&= R_A + R_B + \text{Hash}(R||m) \times (Q_A + Q_B) \\
&= (R_A + \text{Hash}(R||m) \times Q_A) + (R_B + \text{Hash}(R||m) \times Q_B)
\end{align*}

Then $P=P'$ only holds if each of the individual signatures are correct. This trivially generalizes from 2-of-2 to any number of signers m.

Note that this version is actually insecure, and the full correct version can be found in the [MuSig Paper](https://bitcoinops.org/en/topics/musig/). Like the naive version, the full version also uses the linearity behind Schnorr to combine the public keys, nonces, and signatures into single values which act identically to a single sig.

# [BIP 340](https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki)

This BIP is long and dense, so let's start with the [design](https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki#Design) section:

> ### Design
>Schnorr signature variant Elliptic Curve Schnorr signatures for message m and public key P generally involve a point R, integers e and s picked by the signer, and the base point G which satisfy e = hash(R || m) and s⋅G = R + e⋅P. Two formulations exist, depending on whether the signer reveals e or R: 

The (R,s) algorithm described in this notebook is only one of two possible implementations. The BIP explores both, discusses the various trade-offs, and picks the one which shares (R,s). Let's see why that is.

> 1. Signatures are pairs (e, s) that satisfy e = hash(s⋅G - e⋅P || m). This variant avoids minor complexity introduced by the encoding of the point R in the signature (see paragraphs "Encoding R and public key point P" and "Implicit Y coordinates" further below in this subsection). Moreover, revealing e instead of R allows for potentially shorter signatures: Whereas an encoding of R inherently needs about 32 bytes, the hash e can be tuned to be shorter than 32 bytes, and a short hash of only 16 bytes suffices to provide SUF-CMA security at the target security level of 128 bits. However, a major drawback of this optimization is that finding collisions in a short hash function is easy. This complicates the implementation of secure signing protocols in scenarios in which a group of mutually distrusting signers work together to produce a single joint signature (see Applications below). In these scenarios, which are not captured by the SUF-CMA model due its assumption of a single honest signer, a promising attack strategy for malicious co-signers is to find a collision in the hash function in order to obtain a valid signature on a message that an honest co-signer did not intend to sign.
> 2. Signatures are pairs (R, s) that satisfy s⋅G = R + hash(R || m)⋅P. This supports batch verification, as there are no elliptic curve operations inside the hashes. Batch verification enables significant speedups.[4]

> Since we would like to avoid the fragility that comes with short hashes, the e variant does not provide significant advantages. We choose the R-option, which supports batch verification. 

## Description of the e-variant

The e-variant works as follows.

Sign:
1. Generate random number $k$
2. Calculate $R=k \times G$
3. Calculate $s = k + \text{Hash}(R||m) \cdot d_A$
4. Broadcast $(e,s)$

Verify:
1. Calculate $e'=\text{Hash}(s \cdot G - e \cdot P)$
3. Check that $e=e'$

## Advantages of the e-variant

### Simpler Implementation
Because (e,s) are just two numbers, whereas (R,s) is an elliptic curve point and a number, it's easier to encode (e,s).
However there's known ways of dealing with the implementation complexity of encoding a point, so it's not a huge deal.

### Shorter Signatures
In bitcoin, all signatures must be saved to the blockchain, so shorter signatures allow for more transactions per block and lower fees per transaction. The table below compares the space used by the e and R variants:

| variant | signature | size of R or e | size of s | total size |
|---------|-----------|----------------|-----------|------------|
| e-variant | (e,s) | 16 bytes | 32 bytes | 48 bytes |
| R-variant | (R,s) |32 bytes | 32 bytes | 64 bytes |


Signatures in the e-variant are 25% shorter!

However, shorter signatures introduces another problem from [hash collision attacks](https://medium.com/blockstream/insecure-shortcuts-in-musig-2ad0d38a97da). Suppose an honest party were to give their signing nonce $R$ to a dishonest party before they knew what message $m$ they were going to sign. The dishonest party could then generate lots of hashes for two different types of messages $e_0=\text{hash}(R||m_0)$ and $e_1=\text{hash}(R||m_1)$ until they find a collision $e_0=e_1$. Then they give the honest party $m_0$ whose signature $s$ is also valid on $m_1$.

For a hash with $N$ bits, a naive birthday attack would cut the number of steps for finding a collision down to $\sqrt{N}$, for example by comparing every $e_0$ with every $e_1$. However, by opening many parallel sessions with the honest signer and obtaining many nonces, a dishonest signer can use [Wagner's Algorithm](https://www.iacr.org/archive/crypto2002/24420288/24420288.pdf) to find a collision in even shorter time. Therefore it might be preferable to use the longer signatures with the e-variant, but then this advantage goes away.

Note that I think the proper implementation of Schnorr does not allow sharing the nonce $R$ before knowing the message $m$, but I'm guessing this is mentioned as a potential concern in order to err on the side of safety. This is a tempting optimization to make in lightning due to reducing the number of back-and-forth messages during updates.

## Advantages of the R-variant

### Batch Processing

The R-variant allows for an algorithm called batch processing, in which *different* messages and signatures can be aggregated during the verification process as an optimization step. Like multi-sig, this is possible because of the linearity of Schnorr. For example, blocks with thousands of signatures can be verified much more efficiently.



## Key-Prefixing

From the BIP:

> **Key prefixing** Using the verification rule above directly makes Schnorr signatures vulnerable to "related-key attacks" in which a third party can convert a signature (R, s) for public key P into a signature (R, s + a⋅hash(R || m)) for public key P + a⋅G and the same message m, for any given additive tweak a to the signing key. This would render signatures insecure when keys are generated using BIP32's unhardened derivation and other methods that rely on additive tweaks to existing keys such as Taproot.

The hash mentioned above, $\text{Hash}(R||m)$, is insecure when signed by a party with two public keys in which the private keys are offset by some number $a$ known by the attacker. This is actually the case if the attacker knows your xpub, as described in BIP32.

The attack works as follows. Suppose Alice has two addresses in her wallet with public keys $Q_{A1}$ and $Q_{A2}$ and private keys $d_{A1}$ and $d_{A2}$, in which $d_{A2} - d_{A1} = a$.

Suppose Alice sends 1 BTC to Bob from her $Q_{A1}$ address. That transaction corresponds to a message $m$ and signature $(R,s)$ such that $s \cdot G = R + \text{Hash}(R||m) \cdot Q_{A1}$.

Bob can use his knowledge of $a$ to generate a new signature $s'=s + a \cdot \text{Hash}(R||m)$. This will produce a valid signature for $Q_{A2}$, but only for the same message $m$. The math works as follows.

\begin{align*}
s' \times G &= (s + a \cdot \text{Hash}(m||R)) \times G \\
&= s \times G + a \cdot \text{Hash}(m||R) \times G \\
&= R + \text{Hash}(R||m) \times Q_{A1} + a \cdot \text{Hash}(m||R) \times G \\
&= R + (\text{Hash}(R||m) \cdot d_{A1} + a \cdot \text{Hash}(m||R)) \times G \\
&= R + (\text{Hash}(R||m) \cdot (d_{A1} + a)) \times G \\
&= R + (\text{Hash}(R||m) \cdot d_{A2}) \times G \\
&= R + \text{Hash}(R||m) \times Q_{A2}
\end{align*}

A simple fix is to put the public key $Q$ inside the hash as well:

> To protect against these attacks, we choose key prefixed[5] Schnorr signatures which means that the public key is prefixed to the message in the challenge hash input. This changes the equation to s⋅G = R + hash(R || P || m)⋅P. It can be shown that key prefixing protects against related-key attacks with additive tweaks. In general, key prefixing increases robustness in multi-user settings, e.g., it seems to be a requirement for proving the MuSig multisignature scheme secure (see Applications below). 

Now if Bob tried to sign the message $m$ with $Q_{A2}$ he would be unable since it uses a different hash.

But is this even really a big deal? After all, in regular bitcoin transactions the public key is already in the hash implicitly by being part of the message, due to referencing the spending UTXO. If Bob tried to sign the *same* message with $Q_{A2}$, the message would be irrelevant because those funds are locked in a *different* UTXO.

> We note that key prefixing is not strictly necessary for transaction signatures as used in Bitcoin currently, because signed transactions indirectly commit to the public keys already, i.e., m contains a commitment to pk. However, this indirect commitment should not be relied upon because it may change with proposals such as SIGHASH_NOINPUT (BIP118), and would render the signature scheme unsuitable for other purposes than signing transactions, e.g., signing ordinary messages. 

Essentially, there are future plans to add a new SIGHASH to OP_CHECKSIG which would allow signing transactions that don't reference a specific UTXO. For example, if ten different transactions sent to a certain address, then one transaction could spend from all of them as one input, instead of needing to reference ten different inputs, which is 
more space-efficient. This SIGHASH is also needed for eltoo, an improved implementation of the lightning network.

If Alice had signed her transaction with SIGHASH_NOINPUT or SIGHASH_ANYPREVOUT, then Bob could steal from her. For those reasons, the hash used in Schnorr is $\text{Hash}(R||P||m)$.

(Fill this space with work up through Public Key Generation)

## Public Key Conversion

> As an alternative to generating keys randomly, it is also possible and safe to repurpose existing key generation algorithms for ECDSA in a compatible way. The secret keys constructed by such an algorithm can be used as sk directly. The public keys constructed by such an algorithm (assuming they use the 33-byte compressed encoding) need to be converted by dropping the first byte. Specifically, BIP32 and schemes built on top of it remain usable. 

Since both ECDSA and Schnorr are based on elliptic curves, any scheme which creates private/public key pairs in one scheme can be used to create pairs in the other. Specifically, a private key $sk$ is a 32-byte integer, and a public key $pk$ is a point $P=(x,y)$ such that $pk = sk \cdot G$. A lot of work went into BIP32 to allow deterministic derivation of hierarchical trees of public and private keys, so it's nice that it's all easily transferable to Schnorr. The only slight difference is the Schnorr public key removes the y-bit for evenness/oddness.

## Default Signing

> Input:
> - The secret key sk: a 32-byte array
> - The message m: a 32-byte array
> - Auxiliary random data a: a 32-byte array

> The algorithm Sign(sk, m) is defined as:
> - Let d' = int(sk)
> - Fail if d' = 0 or d' ≥ n
> - Let P = d'⋅G
> - Let d = d' if has_even_y(P), otherwise let d = n - d'.

No surprises so far. We can see the transformation to get the actual signing key $d=n-d'$ when $y(d' \cdot G)$ is even, as eluded to above.

> - Let t be the byte-wise xor of $bytes(d)$ and $hash_{BIP0340/aux}(a)$[11].
> - Let $rand = hash_{BIP0340/nonce}(t || bytes(P) || m)$[12].
> - Let $k' = int(rand) \mod n$[13].

Basically we want to be *really* sure that the one-time nonce $k$ is as random and unpredictable as possible. The first line makes sure there's no unintended correlations between the private key $d$ and $k$. Since $s = k + e \cdot d$, it might be possible to solve for $k$ and $d$ given $s$, $e$, and some known relation between $k$ and $d$. This is mentioned in footnote 11.

The second line is another example of erring on the side of safety. Basically if an attacker somehow managed to trick you into signing the same message $m$ with the same nonce $k$ but a different hash $e'$ (for example by getting you to generate $e$ with the wrong public key $P'$) then they could steal your private key:
\begin{align*}
s &= k + e \cdot d \\
s' &= k + e' \cdot d \\
s - s' &= (e - e') \cdot d \\
d &= (s - s') \cdot (e - e') ^ {-1}
\end{align*}
Mixing the public key into the nonce would safeguard against such an attack. The footnote 12 points to a reference which goes more in depth [here](https://moderncrypto.org/mail-archive/curves/2020/001012.html).

The third line's footnote talks about how taking a random 32-byte integer mod n would create a biased result. What I think this means is if a number $k$ is generated such that $n<rand<2^{256}$ then it wraps around, so numbers between $0<d<2^{256}-n$ occur twice as often as they should. Not really sure what the ramifactions would be, intuitively I would think this to be equivalent to reducing the security from $2^{256}$ bits to $\log_2(n)$ which wouldn't seem that bad even if n were, say, half of p, so there might be some other attack I'm unaware of. In any case, p and n are very close (both around $2^{256}$), so it ends up not being a problem.

> - Fail if k' = 0.
> - Let R = k'⋅G.
> - Let k = k' if has_even_y(R), otherwise let k = n - k' .
> - Let e = int(hashBIP0340/challenge(bytes(R) || bytes(P) || m)) mod n.
> - Let sig = bytes(R) || bytes((k + ed) mod n).

All pretty straightforward.

> - If Verify(bytes(P), m, sig) (see below) returns failure, abort[14].
> - Return the signature sig.

The check at the end is just some extra caution which can prevent certain attacks.

> The auxiliary random data should be set to fresh randomness generated at signing time, resulting in what is called a synthetic nonce. Using 32 bytes of randomness is optimal. If obtaining randomness is expensive, 16 random bytes can be padded with 16 null bytes to obtain a 32-byte array. If randomness is not available at all at signing time, a simple counter wide enough to not repeat in practice (e.g., 64 bits or wider) and padded with null bytes to a 32 byte-array can be used, or even the constant array with 32 null bytes. Using any non-repeating value increases protection against [fault injection attacks](https://moderncrypto.org/mail-archive/curves/2017/000925.html).

So apparently setting $a$ to all zeros is in-theory "secure" and makes the nonce deterministic, but in-practice could allow an attacker to do a fault-injection attack. In a fault-injection attack, the attacker might cause some normal CPU operation to give a different result than expected. If signing the same message twice, this would re-use the same nonce which is dangerous if, say, the fault caused the hash to be calculated incorrectly the second time.

> Using unpredictable randomness additionally increases protection against other side-channel attacks, and is recommended whenever available. Note that while this means the resulting nonce is not deterministic, the randomness is only supplemental to security. The normal security properties (excluding side-channel attacks) do not depend on the quality of the signing-time RNG. 

So even though the nonce isn't deterministic, it doesn't need to be.

### Alternative Signing

> It should be noted that various alternative signing algorithms can be used to produce equally valid signatures. The 32-byte rand value may be generated in other ways, producing a different but still valid signature (in other words, this is not a unique signature scheme). No matter which method is used to generate the rand value, the value must be a fresh uniformly random 32-byte string which is not even partially predictable for the attacker. For nonces without randomness this implies that the same inputs must not be presented in another context. This can be most reliably accomplished by not reusing the same private key across different signing schemes. For example, if the rand value was computed as per RFC6979 and the same secret key is used in deterministic ECDSA with RFC6979, the signatures can leak the secret key through nonce reuse. 

I looked at the last part and tried to figure it out for myself. Suppose we signed the same message $m$ using the same nonce $k$ and private key $d$ in both ECDSA and Schnorr producing $s_1$ and $s_2$. How would an attacker get the private key?

Signatures:
\begin{align*}
s_1 &= (m+rd)k^{-1} \\
s_2 &= k+ed
\end{align*}

Eliminate $k$:
\begin{align*}
k &= s_2 - ed \\
s_1^{-1} &= k(m+rd)^{-1} \\
s_1^{-1} &= (s_2-ed)(m+rd)^{-1}
\end{align*}

Solve for d:
\begin{align*}
s_1^{-1}(m+rd)&=s_2-ed \\
s_1^{-1}m + s_1^{-1}rd &= s_2 - ed \\
s_1^{-1}rd + ed &= s_2 - s_1^{-1}m \\
d(s_1^{-1}r + e) &= s_2 - s_1^{-1}m \\
d &= (s_2 - s_1^{-1})(s_1^{-1}r+e)^{-1}
\end{align*}

So in other words, if you're going to "roll your own crypto" (WHICH YOU SHOULD NEVER DO), make sure that none of the nonces could ever be re-used in any context, including completely separate signature schemes. Since this implementation of $rand$ incorporates the private key, the public key, some random noise, and two separate hashes with their own tags, it should be pretty hard to re-use the nonce in another context.

> **Nonce exfiltration protection** It is possible to strengthen the nonce generation algorithm using a second device. In this case, the second device contributes randomness which the actual signer provably incorporates into its nonce. This prevents certain attacks where the signer device is compromised and intentionally tries to leak the secret key through its nonce selection. 

I did some google searches and found a [Blockstream article](https://medium.com/blockstream/anti-exfil-stopping-key-exfiltration-589f02facc2e) and a [bitcoin-dev email](https://lists.linuxfoundation.org/pipermail/bitcoin-dev/2020-March/017667.html). The basic idea is if the hardware wallet is compromised, it may sign transactions with bad nonces which are known or predictable to a third party. To protect against this, the software wallet may send random data to the hardware wallet to incorporate into the nonce, and verify that it was actually used. But the software wallet may also be malicious and try to send non-random data to the hardware wallet, so the order of messages and how things get revealed is important. One example of a secure algorithm is the following:

1. Software wallet generates random string $t$ and hash $H_t=\text{hash}(t)$, sends $H_t$ to software wallet.
2. Hardware wallet generates secret $k'$ and $R'=k' \times G$, sends $R$ to software wallet.
3. Software wallet sends $t$ to hardware wallet.
4. Hardware wallet verifies that $t=\text{hash}(t)$.
5. Hardware wallet computes $k=k'+t$, $R=k \times G$, and signs $m$ with $R$, revealing $s$ and $R$ to software wallet.
6. Software wallet verifies signature and also that $R=R'+t \times G$.

> **Multisignatures** This signature scheme is compatible with various types of multisignature and threshold schemes such as MuSig, where a single public key requires holders of multiple secret keys to participate in signing (see Applications below). **It is important to note that multisignature signing schemes in general are insecure with the rand generation from the default signing algorithm above (or any other deterministic method).**

I might look at the MuSig paper later, so ignore this section for now.

> **Precomputed public key data** For many uses the compressed 33-byte encoding of the public key corresponding to the secret key may already be known, making it easy to evaluate has_even_y(P) and bytes(P). As such, having signers supply this directly may be more efficient than recalculating the public key from the secret key. However, if this optimization is used and additionally the signature verification at the end of the signing algorithm is dropped for increased efficiency, signers must ensure the public key is correctly calculated and not taken from untrusted sources. 

I think this is pretty self-explanatory